# TellusからGCOM-C/SSTで海面温度を取得し、シーンのファイルをダウンロード
- ダウンロードする年月を、それぞれリストで定義
- ダウンロードする拡張子をリストで定義
- 年月のフォルダ内にダウンロード
- すべてダウンロードしたらZipで圧縮

## 必要なパッケージをインポート

In [ ]:
import requests
import json
import os
import pandas as pd
import shutil
import datetime
import time

## 各設定を定数で定義

In [ ]:
# APIトークン
TOKEN = 'ここにAPIトークンを入力'

In [ ]:
headers = {
    "Authorization": "Bearer " + TOKEN,
    'content-type': 'application/json'
}

In [ ]:
URL_BASE = 'https://www.tellusxdp.com/api/traveler/v1'

In [ ]:
# GCOM-C/SST 8日平均 （高次プロダクト）
DATASET_ID = '000eb404-1f69-4735-a966-2f3115269ee3'

In [ ]:
# ダウンロードする月
MONTHS = [
    6,
    7,
    8,
]

In [ ]:
# ダウンロードする年
YEARS = [
    2018,
    2019,
    2020,
    2021,
    2022,
    2023,
]

In [ ]:
# ダウンロードする拡張子
DL_EXTS = ['.tif',]
# DL_EXTS = ['.tif', 'h5',] # h5ファイルもダウンロードする場合はこちら

In [ ]:
# ダウンロード先ディレクトリのベース
DL_DIR_BASE = 'gcom-c-sst'

In [ ]:
# ダウンロード先ディレクトリ（ベースに日時を付与）
now = datetime.datetime.now()
dt = now.strftime('%Y%m%d-%H%M%S')
dl_dir = '{}_{}'.format(DL_DIR_BASE, dt,)

In [ ]:
# 連続してAPIを実行する際にスリープする秒数
API_SLEEP_SEC = 2

## 海面温度（GCOM-C/SST 8日平均）のデータセットを取得

In [ ]:
url = '{}/datasets/{}'.format(URL_BASE, DATASET_ID,)

In [ ]:
query = {}

In [ ]:
payloads = {
    'query': query,
}

In [ ]:
res = requests.get(url, headers=headers, data=json.dumps(payloads))

In [ ]:
res.json()

## シーン検索

In [ ]:
url = '{}/datasets/{}/data-search/'.format(URL_BASE, DATASET_ID,)

In [ ]:
data_list = []

for year in YEARS:
    for month in MONTHS:
        start_datetime = '{}-{}-01T00:00:00Z'.format(year, str(month).zfill(2),)
        end_datetime = '{}-{}-01T00:00:00Z'.format(year, str(month + 1).zfill(2),)

        # print(start_datetime, end_datetime,)

        query = {
            'start_datetime': {
                'gte': start_datetime,
            },
            'end_datetime': {
                'lt': end_datetime,
            },
        }

        payloads = {
            'query': query,
        }

        res = requests.post(url, headers=headers, data=json.dumps(payloads))
        j = res.json()
        features = j['features']

        for feature in features:
            data_id = feature['id']
            data_list.append((str(year), str(month).zfill(2), data_id,))

In [ ]:
data_list[:5]

## ファイル情報一覧

In [ ]:
download_list = []

for year, month, data_id in data_list:
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data/{}/files/'.format(DATASET_ID, data_id,)

    query = {}
    payloads = {
        'query': query,
    }

    res = requests.get(url, headers=headers, data=json.dumps(payloads))

    for f in res.json()['results']:
        file_name = f['name']
        file_id = f['id']

        for ext in DL_EXTS:
            if file_name.endswith(ext):
                download_list.append((year, month, data_id, file_name, file_id,))

In [ ]:
download_list[:5]

## シーンのファイルをダウンロード

In [ ]:
download_urls = []

for year, month, data_id, file_name, file_id, in download_list:
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data/{}/files/{}/download-url/'.format(DATASET_ID, data_id, file_id,)

    query = {}
    payloads = {
        'query': query,
    }

    res = requests.post(url, headers=headers, data=json.dumps(payloads))

    if 'download_url' in res.json():
        url = res.json()['download_url']
        print('Info : download_url : {} (year : {}, month : {}, data_id : {}, file_name : {}, file_id : {})'.format(url, year, month, data_id, file_name, file_id,))

        download_urls.append((year, month, data_id, file_name, file_id, url,))
    else:
        print('Warning : download_url does not exist in response (year : {}, month : {}, data_id : {}, file_name : {}, file_id : {})'.format(year, month, data_id, file_name, file_id,))
        print(res)
        print(res.json())

    time.sleep(API_SLEEP_SEC)

In [ ]:
download_urls[:5]

In [ ]:
for year, month, data_id, file_name, file_id, url, in download_urls:
    download_dir = '{}/{}-{}'.format(dl_dir, year, month,)
    download_file_path = '{}/{}'.format(download_dir, file_name,)

    if not os.path.isdir(download_dir):
        os.makedirs(download_dir)

    print('download : {}'.format(download_file_path,))

    download_res = requests.get(url, stream = True)

    with open(download_file_path, 'wb') as f:
        for chunk in download_res.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()

    time.sleep(API_SLEEP_SEC)

## ダウンロードしたファイルをzip圧縮

In [ ]:
zip_file_name = '{}.zip'.format(dl_dir,)

shutil.make_archive(dl_dir, format='zip', root_dir='.', base_dir=dl_dir,)

In [ ]:
shutil.rmtree(dl_dir,)